In [1]:
 import hashlib
 import datetime
 import random

In [2]:
from datetime import datetime

In [3]:
class MerkleTree:
    @staticmethod
    def get_merkle_root(transactions):
        if not transactions:
            return hashlib.sha256("".encode()).hexdigest()
        
        # Ensure the number of transactions is a power of 2
        while not (len(transactions) & (len(transactions) - 1) == 0):
            transactions.append("")
        
        # Convert transactions to strings and hash them
        leafs = [hashlib.sha256(str(tx).encode()).hexdigest() for tx in transactions]
        
        # Build the Merkle tree
        while len(leafs) > 1:
            next_level = []
            for i in range(0, len(leafs), 2):
                combined_hash = hashlib.sha256((leafs[i] + leafs[i + 1]).encode()).hexdigest()
                next_level.append(combined_hash)
            leafs = next_level
        
        # The last remaining hash is the Merkle root
        return leafs[0]


In [4]:
class Block:
    def __init__(self, index, transactions, previous_hash):
        self.index = index
        self.timestamp = datetime.now()
        self.transactions = transactions
        self.previous_hash = previous_hash
        self.nonce = 0
        self.merkle_root = MerkleTree.get_merkle_root(transactions)
        self.hash = self.create_hash()

    def create_hash(self):
        block_data = f"{self.index}{self.timestamp}{self.merkle_root}{self.previous_hash}{self.nonce}"
        return hashlib.sha256(block_data.encode()).hexdigest()

    def mine_block(self, difficulty):
        mining = True  # Flag for the mining loop
        target = "0" * difficulty  # Target string with leading zeros
        while mining:
            # Calculate hash with the current nonce
            self.hash = self.create_hash()
            # Check if the hash meets the difficulty requirement
            if self.hash[:difficulty] == target:
                mining = False  # Mining successful
                print('MINED with hash:', self.hash)
            else:
                self.nonce += 1

In [5]:
class VotesBlockchain:
    def __init__(self, difficulty=4, max_transactions=4):
        self.chain = [self.create_genesis_block()]
        self.pending_transactions = []
        self.difficulty = difficulty
        self.max_transactions = max_transactions

    def create_genesis_block(self):
        return Block(0, ["Genesis Block"], "0")

    def get_latest_block(self):
        return self.chain[-1]

    def add_transaction(self, voter_name, candidate_name):
        """
        Add a single transaction.
        """
        transaction = (voter_name, candidate_name)
        if len(self.pending_transactions) >= self.max_transactions:
            print("Transaction pool is full! Please mine a block.")
        else:
            self.pending_transactions.append(transaction)

    def add_transactions(self, transactions):
        """
        Add multiple transactions as a list.
        """
        for voter_name, candidate_name in transactions:
            self.add_transaction(voter_name, candidate_name)

    def mine_pending_transactions(self):
        """
        Mine transactions in chunks, limited by max_transactions.
        """
        if len(self.pending_transactions) == 0:
            print("No transactions to mine.")
            return

        transactions_to_mine = self.pending_transactions[:self.max_transactions]
        new_block = Block(
            index=len(self.chain),
            transactions=transactions_to_mine,
            previous_hash=self.get_latest_block().hash
        )
        new_block.mine_block(self.difficulty)
        self.chain.append(new_block)
        self.pending_transactions = self.pending_transactions[self.max_transactions:]

    def is_chain_valid(self):
        """
        Validate the integrity of the blockchain.
        """
        previous_block_hash = "0"
        for i in range(len(self.chain)):
            current_block = self.chain[i]
            print(f"This block hash: {current_block.hash}")
            if previous_block_hash != "0":
                print(f"Saved previous hash: {current_block.previous_hash}")
                is_valid = previous_block_hash == current_block.previous_hash
                print(f"Was not tampered with: {is_valid}")
                if not is_valid:
                    return False
            previous_block_hash = current_block.hash
            print("MOVING TO THE NEXT -----------------------------------------------------------")

        return True

In [6]:
my_blockchain = VotesBlockchain()

In [7]:
transactions = [
    ('Alice Brown', 'Candidate A'),
    ('Bob Smith', 'Candidate B'),
    ('Charlie Johnson', 'Candidate C'),
    ('Diana Green', 'Candidate A')
]
my_blockchain.add_transactions(transactions)

In [8]:
print("Mining block...")
my_blockchain.mine_pending_transactions()

Mining block...
MINED with hash: 000040148ba387439227a672974373eed46a908f242a7124d641cdc2343b2789


In [9]:
print("Is blockchain valid?", my_blockchain.is_chain_valid())

This block hash: 626e5082de065ac180548960225c29bd45cdb63e5bb1e0f395a9099958846aa1
MOVING TO THE NEXT -----------------------------------------------------------
This block hash: 000040148ba387439227a672974373eed46a908f242a7124d641cdc2343b2789
Saved previous hash: 626e5082de065ac180548960225c29bd45cdb63e5bb1e0f395a9099958846aa1
Was not tampered with: True
MOVING TO THE NEXT -----------------------------------------------------------
Is blockchain valid? True


In [10]:
my_blockchain.add_transaction('Hank White', 'Candidate A')

In [11]:
my_blockchain.add_transaction('Anna Blue', 'Candidate B')

In [12]:
print("Pending transactions before mining:", my_blockchain.pending_transactions)

Pending transactions before mining: [('Hank White', 'Candidate A'), ('Anna Blue', 'Candidate B')]


In [13]:
print("Mining block...")
my_blockchain.mine_pending_transactions()

Mining block...
MINED with hash: 00006d86a73bb2d1ead967d531a03b93025a1ec5de6e5c13880a0dc4ffc022a8


In [14]:
print("Pending transactions after mining:", my_blockchain.pending_transactions)

Pending transactions after mining: []
